Czy uważasz, że trzeci proces zapisujący dane jest zbędny i może zostać zastąpiony threading, tak żeby istniały dwa procesy: jeden nasłuchujący, drugi przetwarzający i zapisujący, gdzie przetwarzanie i zapisywanie działają na osobnych wątkach, aby nie blokować się wzajemnie? Czy lepiej wykorzystać trzy procesy tak jak w powyższym przykładzie?

Oba podejścia mają swoje zalety i zależą od konkretnej sytuacji. Oto krótkie porównanie:

Wykorzystanie trzech procesów:

Zalety:
Każdy proces działa niezależnie od siebie, co oznacza, że przetwarzanie danych i zapisywanie ich do pliku nie blokują procesu nasłuchującego.
Procesy mogą wykorzystywać pełne korzyści z wielowątkowości dostępnej w ramach multiprocessing, tak jak w przykładzie z poprzedniej odpowiedzi.
Wady:
Istnieje dodatkowy narzut związany z komunikacją międzyprocesową poprzez kolejki, co może wpływać na wydajność w przypadku dużych ilości danych.
Wykorzystanie dwóch procesów i wątków:

Zalety:
Zmniejszony narzut komunikacji międzyprocesowej, ponieważ przetwarzanie danych i zapisywanie do pliku odbywają się w ramach jednego procesu.
Wykorzystanie wielowątkowości w ramach jednego procesu, co może być wydajne dla operacji przetwarzania danych i zapisywania ich do pliku.
Wady:
Konieczność użycia modułu threading zamiast multiprocessing. Może to wymagać dodatkowych zabiegów, takich jak odpowiednia synchronizacja wątków.
W przypadku dużych obciążeń obliczeniowych lub zapisu do pliku, może wystąpić blokowanie jednego wątku przez drugi, co wpływa na wydajność.
Ostateczny wybór zależy od specyfiki Twojego programu, wymagań wydajnościowych i preferencji. Jeśli oczekujesz dużej ilości danych lub potrzebujesz maksymalnej niezależności między procesami, trzy procesy mogą być lepszym rozwiązaniem. Jeśli jednak zależy Ci na zminimalizowaniu komunikacji międzyprocesowej i wykorzystaniu wielowątkowości w ramach jednego procesu, dwie osiowe i wątki mogą być bardziej odpowiednie.

In [ ]:
import serial
import multiprocessing
import threading

# Funkcja odbierająca dane z portu szeregowego i umieszczająca je w kolejce
def odbierz_dane(port, kolejka):
    while True:
        dane = port.readline().decode().strip()
        if dane:
            kolejka.put(dane)

# Funkcja przetwarzająca dane odebrane z kolejki i wyświetlająca je na bieżąco
def przetwarzaj_dane(kolejka):
    def zapisz_do_pliku():
        with open("dane.txt", "a") as plik:
            while True:
                if not kolejka.empty():
                    dane = kolejka.get()
                    plik.write(dane + "\n")

    # Inicjalizacja wątku zapisującego dane do pliku
    watek_zapisujacy = threading.Thread(target=zapisz_do_pliku)
    watek_zapisujacy.daemon = True
    watek_zapisujacy.start()

    while True:
        if not kolejka.empty():
            dane = kolejka.get()
            # Tutaj możesz wykonywać przetwarzanie na danych
            przetworzone_dane = dane.upper()
            print("Przetworzone dane:", przetworzone_dane)

# Inicjalizacja portu szeregowego
port = serial.Serial('/dev/ttyUSB0', 9600, timeout=1)

# Inicjalizacja kolejki danych
kolejka = multiprocessing.Queue()

# Proces nasłuchujący dane z portu szeregowego
proces_nasluchujacy = multiprocessing.Process(target=odbierz_dane, args=(port, kolejka))
proces_nasluchujacy.daemon = True
proces_nasluchujacy.start()

# Proces przetwarzający dane z kolejki
proces_przetwarzajacy = multiprocessing.Process(target=przetwarzaj_dane, args=(kolejka,))
proces_przetwarzajacy.daemon = True
proces_przetwarzajacy.start()

# Główna pętla programu
while True:
    # Reszta logiki programu
    # ...

    # Tu możesz wykonywać inne operacje, które nie blokują procesów


W tym przykładzie wykorzystuję moduły multiprocessing i threading do uruchomienia dwóch procesów oraz jednego wątku. Proces proces_nasluchujacy odpowiada za nasłuchiwanie danych z portu szeregowego i umieszczanie ich w kolejce. Proces proces_przetwarzajacy przetwarza dane z kolejki i wyświetla je na bieżąco. Wątek watek_zapisujacy zapisuje dane z kolejki do pliku.

W głównej pętli programu można dodawać logikę lub inne operacje, które nie blokują działania procesów i wątków

W tym poprawionym przykładzie wątek watek_zapisujacy został przeniesiony do funkcji przetwarzaj_dane, która jest wywoływana w drugim procesie proces_przetwarzajacy. Teraz proces przetwarzający będzie zarówno przetwarzał dane, jak i zapisywał je do pliku poprzez wątek.

Przepraszam za niejasność i mam nadzieję, że ten przykład spełnia Twoje oczekiwania.